In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from dataclasses import dataclass
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dataclass
class LanguageContext:
    user_language: str = "English"

@dynamic_prompt
def user_language_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on user role."""
    user_language = request.runtime.context.user_language
    base_prompt = "You are a helpful assistant."

    if user_language != "English":
        return f"{base_prompt} only respond in {user_language}."
    elif user_language == "English":
        return base_prompt

In [ ]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

MODEL_NAME = "gemini-2.5-flash-lite"

model = init_chat_model(model=MODEL_NAME, model_provider="google_genai")

agent = create_agent(
    model=model,
    context_schema=LanguageContext,
    middleware=[user_language_prompt]
)

In [ ]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="German")
)

print(response["messages"][-1].content)

In [ ]:
response = agent.invoke(
    {"messages": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="Spanish")
)

print(response["messages"][-1].content)

In [ ]:
response = agent.invoke(
    {"messages": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="French")
)

print(response["messages"][-1].content)